### Main goals:
#### 1. Create a unique ID for each contractor by using the "recipient parent name" variable. The ID will be used to aggregate individual contract records by contractor by year. The unique ID is created by removing special characters, numbers, common words, and spaces. 
#### 2. The aggregated federal contract award amounts ("federal_action_obligation") are the cross checked with a dataset with the annual federal contract award amount of the top 100 contractors prepared by the GSA.
#### 3. There is a variable with recipient's duns number in the data set, but from eyeballing the data, there are a lot of missing/incorrect entries. I first tried to aggregate by the duns variable and shows that the values are way off.
#### 4. The aggregated award amounts created by the unique ID created matched fairly well with the GSA data set.

In [2]:
import pandas as pd
import re

## First try by duns... clearly doesn't work

In [3]:
def f(x):
     return pd.Series(dict(federal_action_obligation = x['federal_action_obligation'].sum(),
                           recipient_name = x['recipient_parent_name'].unique()[0]))



year = range(2006, 2007)
table = [4]
contractors = []
for i, y in enumerate(year):
    for j in range(1, table[i]+1):
        df = pd.read_csv(f"/Users/ptw/Downloads/{y}/{y}_all_Contracts_Full_20190613_{j}.csv",
                         usecols=['recipient_parent_duns','recipient_parent_name','federal_action_obligation'])
        fc_g = df.groupby(['recipient_parent_duns']).apply(f)
fc_g
fc_g.to_csv('2006_by_duns.csv')

/Users/ptw/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,federal_action_obligation,recipient_name
recipient_parent_duns,,
0.0,199565.41,S.N.C. SCIONTI
77743.0,210343.00,SPOUSES OF SERVICE PERSONNEL
231233.0,4958.00,BONNEY'S AG & AUTO REPAIR
816355,651.67,J-MAR PROPERTIES
1000363,0.00,LOUIS M. GERSON CO. INC.
1000947.0,4485.00,"STITCH THIS, LLC"
1001049,2633.00,SPINELLI RAVIOLI MFG CO INC
1002229,54875.22,NORTH ATLANTIC FISH CO. INC.
1002781,12135.00,TRIANGLE ENGINEERING INC.


## By cleaned recipient parent name column

In [92]:
df2 = pd.read_csv('most_common_words_cleaned.csv')
mcw = list(df2['names'])

In [93]:
df3 = pd.read_csv('top100_cleaned.csv')

In [94]:
top100 = df3['0']

In [95]:
top100c = [re.sub(r'\W+', '', x) for x in top100]

In [125]:
def f(x):
    return pd.Series(dict(federal_action_obligation=x['federal_action_obligation'].sum(),
                          award_count=x['recipient_parent_name'].count(),
                          recipient_name=x['recipient_parent_name'].unique()[0]))

In [126]:
def g(x):
    return pd.Series(dict(federal_action_obligation=x['federal_action_obligation'].sum(),
                          award_count=x['award_count'].sum(),
                          recipient_name=x['recipient_name'].unique()[0]))

In [99]:
def wsplit(x):
    cw = x.split()
    resultwords = []
    for word in cw:
        if word not in mcw:
            resultwords.append(word)
    result = ' '.join(resultwords)
    return result

In [100]:
def rep100(x):
    for c in top100c:
        if c in x:
            x = c
    return x

In [127]:
#save cleaned name id in raw data csv files
year = range(2001, 2019)
table = [1, 1, 2, 2, 3, 4, 5, 5, 4, 4, 4, 4, 3, 3, 5, 5, 5, 6]

for i, y in enumerate(year):
    for j in range(1, table[i]+1):
        df = pd.read_csv(f"/Users/ptw/Downloads/{y}/{y}_all_Contracts_Full_20190613_{j}.csv")
        df['recipient_parent_name'].fillna(df['recipient_name'], inplace=True)
        df.dropna(subset=['recipient_parent_name'], inplace=True)
        df['recipient_parent_name1'] = df['recipient_parent_name'].map(
            lambda x: re.sub(r"\([^)]*\)", '', x.lower()))
        df['recipient_parent_name1'] = df['recipient_parent_name1'].map(
            lambda x: re.sub(r"[$+,:;=?@#|'<>.^*()%!¬ø√¢/]", '', x))
        df['recipient_parent_name1'] = df['recipient_parent_name1'].map(
            lambda x: re.sub(r"[&-]", ' ', x))
        df['no_commonwords'] = df.recipient_parent_name1.apply(wsplit)
        df['clean_name'] = df['no_commonwords'].map(lambda x: re.sub(r'\W+', '', x))
        df['clean_name_100'] = df['clean_name'].map(rep100)
        df.loc[df['clean_name_100'] == 'internationalbusinessm', 'clean_name_100'] = 'ibm'
        df.loc[df['clean_name_100'] == 'federalexpress', 'clean_name_100'] = 'fedex'
        df.drop(['no_commonwords','clean_name'], axis=1, inplace=True)
        df.to_csv(f"/Users/ptw/Downloads/{y}/{y}_{j}_FC_raw_id.csv")


/Users/ptw/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4,7,32,33,38,41,43,51,53,54,61,77,91,92,94,100,113,129,130,148,171,172,211,264,266,268,270,272) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ptw/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4,7,32,33,38,41,43,51,53,54,61,77,85,86,91,92,93,94,100,113,129,148,157,158,167,168,171,172,211,264,266,268,270,272) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ptw/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4,7,32,33,38,41,43,51,53,54,61,77,91,92,94,100,113,129,148,171,172,211,264,266,268,270,272) have mixed types. Specify dtype option on import or set low_memory=False.
  interact

/Users/ptw/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (34,37,39,49,50,57,75,76,89,138) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ptw/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3,37,39,49,50,57,75,76,89) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ptw/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3,37,39,49,50,57,66,75,76,89) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ptw/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3,34,37,39,43,49,50,57,66,75,76,89,109,138) have m

In [ ]:
#For collapsing raw data
year = range(2001, 2019)
table = [1, 1, 2, 2, 3, 4, 5, 5, 4, 4, 4, 4, 3, 3, 5, 5, 5, 6]
fc_all = pd.DataFrame()
for i, y in enumerate(year):
    for j in range(1, table[i]+1):
        df = pd.read_csv(f"/Users/ptw/Downloads/{y}/{y}_all_Contracts_Full_20190613_{j}.csv",
                         usecols=['recipient_name', 'recipient_parent_duns', 'recipient_parent_name', 'federal_action_obligation'])
        df['recipient_parent_name'].fillna(df['recipient_name'], inplace=True)
        df.dropna(subset=['recipient_parent_name'], inplace=True)
        df['recipient_parent_name1'] = df['recipient_parent_name'].map(
            lambda x: re.sub(r"\([^)]*\)", '', x.lower()))
        df['recipient_parent_name1'] = df['recipient_parent_name1'].map(
            lambda x: re.sub(r"[$+,:;=?@#|'<>.^*()%!¬ø√¢/]", '', x))
        df['recipient_parent_name1'] = df['recipient_parent_name1'].map(
            lambda x: re.sub(r"[&-]", ' ', x))
        df['no_commonwords'] = df.recipient_parent_name1.apply(wsplit)
        df['clean_name'] = df['no_commonwords'].map(lambda x: re.sub(r'\W+', '', x))
        df['clean_name_100'] = df['clean_name'].map(rep100)
        new_df = df.groupby(['clean_name_100']).apply(f)
        fc_all = pd.concat([fc_all, new_df])


In [88]:
fc_sum = fc_all.groupby(['clean_name_100']).apply(g)
fc_sum.to_csv('2006_by_clean_name.csv')

In [115]:
df.loc[df['clean_name_100'] == 'internationalbusinessm', 'clean_name_100'] = 'ibm'

In [120]:
df.loc[df['clean_name_100'] == 'federalexpress', 'clean_name_100'] = 'fedex'

In [122]:
df.loc[df['clean_name_100'] == 'fedex']

,federal_action_obligation,recipient_name,recipient_parent_name,recipient_parent_duns,recipient_parent_name1,no_commonwords,clean_name,clean_name_100
3,1038188.00,FEDERAL EXPRESS CHARTER PROGRA,FEDEX CORPORATION,3141970,fedex corporation,fedex,fedex,fedex
1078,21.82,FEDERAL EXPRESS CORPORATION,FEDEX CORPORATION,3141970,fedex corporation,fedex,fedex,fedex
1990,15858.00,FEDERAL EXPRESS CORPORATION,FEDEX CORPORATION,3141970,fedex corporation,fedex,fedex,fedex
2720,-30770.00,FEDERAL EXPRESS CHARTER PROGRA,FEDEX CORPORATION,003141970,fedex corporation,fedex,fedex,fedex
2721,438171.00,FEDERAL EXPRESS CHARTER PROGRA,FEDEX CORPORATION,003141970,fedex corporation,fedex,fedex,fedex
3312,-500.00,FEDERAL EXPRESS CORPORATION,FEDEX CORPORATION,003141970,fedex corporation,fedex,fedex,fedex
3345,114.85,FEDERAL EXPRESS CORPORATION,FEDEX CORPORATION,003141970,fedex corporation,fedex,fedex,fedex
3351,0.00,FEDERAL EXPRESS CORPORATION,FEDEX CORPORATION,003141970,fedex corporation,fedex,fedex,fedex
3588,15000.00,FEDERAL EXPRESS CORPORATION,FEDEX CORPORATION,003141970,fedex corporation,fedex,fedex,fedex
3602,189.69,FEDERAL EXPRESS CORPORATION,FEDEX CORPORATION,003141970,fedex corporation,fedex,fedex,fedex


In [117]:
df.head()

,federal_action_obligation,recipient_name,recipient_parent_name,recipient_parent_duns,recipient_parent_name1,no_commonwords,clean_name,clean_name_100
0,190.50,ESCHENBACH OPTIK AMERICA INCORPORATED,ESCHENBACH OPTIK AMERICA INC,138152665,eschenbach optik america inc,eschenbach optik america,eschenbachoptikamerica,eschenbachoptikamerica
1,53.44,ALIMED INCORPORATED,ALIMED INC.,56007248,alimed inc,alimed,alimed,alimed
2,57.54,SURGICAL APPLIANCE INDUSTRIES INCORPORATED,SURGICAL APPLIANCE INDUSTRIES INC.,4247383,surgical appliance industries inc,surgical appliance industries,surgicalapplianceindustries,surgicalapplianceindustries
3,1038188.00,FEDERAL EXPRESS CHARTER PROGRA,FEDEX CORPORATION,3141970,fedex corporation,fedex,fedex,fedex
4,1095.16,NATIONAL INDUSTRIES FOR THE BLIND,NATIONAL INDUSTRIES FOR THE BLIND,1672120,national industries for the blind,national industries for blind,nationalindustriesforblind,nationalindustriesforblind


In [5]:
df = pd.read_csv(f"/Users/ptw/Downloads/2006/2006_all_Contracts_Full_20190613_4.csv",
                 usecols=['recipient_name', 'recipient_parent_duns', 'recipient_parent_name', 'federal_action_obligation'])

In [27]:
df['recipient_parent_name'].fillna(df['recipient_name'], inplace=True)

In [43]:
df.dropna(subset=['recipient_parent_name'], inplace=True)

In [46]:
df['recipient_parent_name1'] = df['recipient_parent_name'].map(lambda x: re.sub(r"\([^)]*\)", '', x.lower()))

In [47]:
df['recipient_parent_name1'] = df['recipient_parent_name1'].map(
    lambda x: re.sub(r"[$+,:;=?@#|'<>.^*()%!¬ø√¢/]", '', x))

In [49]:
df['recipient_parent_name1'] = df['recipient_parent_name1'].map(
    lambda x: re.sub(r"[&-]", ' ', x))

In [54]:
mcw

['000000000000',
 'and',
 'assn',
 'asso',
 'assoc',
 'associate',
 'associates',
 'association',
 'associations',
 'ave',
 'avenue',
 'blvd',
 'boulevard',
 'co',
 'company',
 'companies',
 'cooperative',
 'cooperatives',
 'corp',
 'corps',
 'corpor',
 'corporat',
 'corporation',
 'corporations',
 'fzco',
 'fze',
 'goverment',
 'government',
 'gp',
 'group',
 'holding',
 'holdings',
 'ii',
 'iii',
 'inc',
 'incorporated',
 'incorporate',
 'int',
 'intl',
 'internati',
 'l l c',
 'l  l  c',
 'l l c',
 'l l c',
 'lc',
 'liability',
 'limited',
 'llc',
 'llp',
 'lp',
 'ltd',
 'mfg',
 'of',
 'pa',
 'prtnr',
 'partners',
 'partnership',
 'pc',
 'plc',
 'pllc',
 'service',
 'services',
 'solution',
 'solutions',
 'street',
 'st',
 'sys',
 'syst',
 'syste',
 'system',
 'systems',
 'the',
 'joint',
 'venture',
 'p a',
 'usa',
 '17474',
 'wireless']

In [55]:
#remove most common words
def wsplit(x):
    cw = x.split()
    resultwords = []
    for word in cw:
        if word not in mcw:
            resultwords.append(word)
    result = ' '.join(resultwords)
    return result

In [56]:
df['no_commonwords'] = df.recipient_parent_name1.apply(wsplit)

In [59]:
df['clean_name'] = df['no_commonwords'].map(lambda x: re.sub(r'\W+', '', x))

In [61]:
df3 = pd.read_csv('top100_cleaned.csv')

In [63]:
top100 = df3['0']

In [65]:
top100c = [re.sub(r'\W+', '', x) for x in top100]

In [76]:
def rep100(x):
    for c in top100c:
        if c in x:
            x = c
    return x

rep100(st)

'fedex'

In [77]:
df['clean_name_100'] = df['clean_name'].map(rep100)

In [78]:
df.head()

,federal_action_obligation,recipient_name,recipient_parent_name,recipient_parent_duns,recipient_parent_name1,no_commonwords,clean_name,clean_name_100
0,190.50,ESCHENBACH OPTIK AMERICA INCORPORATED,ESCHENBACH OPTIK AMERICA INC,138152665,eschenbach optik america inc,eschenbach optik america,eschenbachoptikamerica,eschenbachoptikamerica
1,53.44,ALIMED INCORPORATED,ALIMED INC.,56007248,alimed inc,alimed,alimed,alimed
2,57.54,SURGICAL APPLIANCE INDUSTRIES INCORPORATED,SURGICAL APPLIANCE INDUSTRIES INC.,4247383,surgical appliance industries inc,surgical appliance industries,surgicalapplianceindustries,surgicalapplianceindustries
3,1038188.00,FEDERAL EXPRESS CHARTER PROGRA,FEDEX CORPORATION,3141970,fedex corporation,fedex,fedex,fedex
4,1095.16,NATIONAL INDUSTRIES FOR THE BLIND,NATIONAL INDUSTRIES FOR THE BLIND,1672120,national industries for the blind,national industries for blind,nationalindustriesforblind,nationalindustriesforblind


In [79]:
df.to_csv('2006_cleaned_top100.csv')